In [1]:
import re # 정규표현식을 사용해 텍스트 데이터를 정제하기 위해
import random # 랜덤 숫자를 생성하기 위해
from math import exp, log # 지수함수와 로그함수를 사용하기 위해
from datetime import datetime # 시간을 계산하기 위해
from operator import itemgetter # 키가 아닌 값으로 max, min 값을 구할 때 사용

In [2]:
def clean(s):
    """
        Returns a cleaned, lowercased string
        텍스트 데이터를 정제하고 소문자로 변환해 준다.
    """
    return " ".join(re.findall(r'\w+', s, flags = re.UNICODE)).lower()

In [3]:
def get_data_tsv(loc_dataset,opts):
    """
    Running through data in an online manner
    Parses a tsv file for this competition 
    and yields label, identifier and features
    output:
            label: int, The label / target (set to "1" if test set)
            id: string, the sample identifier
            features: list of tuples, 
                in the form [(hashed_feature_index,feature_value)]

    온라인 학습 방법을 통해 데이터를 실행한다.
    tsv파일을 통해 레이블, identifier, 피처(특성)를 파싱한다.
    결과물:
        label : int, 레이블 / 대상 (테스트 집합 인 경우 "1"로 설정)
        id : 문자열, 샘플 식별자
        features : [(hashed_feature_index, feature_value)] 
                형식의 튜플 목록
    """
    for e, line in enumerate(open(loc_dataset,"rb")):
        if e > 0:
            r = line.decode('utf-8').strip().split("\t")
            id = r[0]

            if opts["clean"]:
                try:
                    r[2] = clean(r[2])
                except:
                    r[1] = clean(r[1])

            # opts["D"] = 2 ** 25 = 33554432
            # Vowpal Wabbit의 해싱트릭을 사용한다.
            # 해싱트릭은 큰 규모의 feature공간을 
            # 고정크기의 표현을 사용해 저장할 수 있게 한다.
            if len(r) == 3: # train set
                features = [(hash(f)%opts["D"],1) for f in r[2].split()]
                label = int(r[1])
            else: #test set
                features = [(hash(f)%opts["D"],1) for f in r[1].split()]
                label = 1

            # bigram을 사용하면 해당 피처[i]와 다음피처[i+1]를 함께 해싱한다.
            if opts["2grams"]:
                for i in range(len(features)-1):
                    features.append(
                        (hash(str(features[i][0])+str(features[i+1][0]))%opts["D"],1))
            yield label, id, features

In [4]:
def dot_product(features,weights):
    """
    Calculate dot product from features and weights
    input:
            features: A list of tuples [(feature_index,feature_value)]
            weights: the hashing trick weights filter, 
            note: length is max(feature_index)
    output:
            dotp: the dot product
    피처(특성)과 가중치로부터 내적을 구한다.
    """
    dotp = 0
    for f in features:
        dotp += weights[f[0]] * f[1]
    return dotp

In [5]:
def train_tron(loc_dataset,opts):
    start = datetime.now()
    print("\nPass\t\tErrors\t\tAverage\t\tNr. Samples\tSince Start")

    # 가중치 초기화
    if opts["random_init"]:
        random.seed(3003)
        weights = [random.random()] * opts["D"]
    else:
        weights = [0.] * opts["D"]

    # Running training passes
    # 학습 실행
    for pass_nr in range(opts["n_passes"]):
        error_counter = 0
        for e, (label, id, features) in enumerate( \
            get_data_tsv(loc_dataset,opts) ):

            # 퍼셉트론은 지도학습 분류기의 일종이다. 
            # 이전 값에 대한 학습으로 예측을 한다. 
            # 내적(dotproduct) 값이 임계 값보다 높거나 낮은지에 따라 
            # 초과하면 "1"을 예측하고 미만이면 "0"을 예측한다.
            dp = dot_product(features, weights) > 0.5

            # 다음 perceptron은 샘플의 레이블을 본다. 
            # 실제 레이블 데이터에서 위 퍼셉트론으로 구한 dp값을 빼준다.
            # 예측이 정확하다면, error 값은 "0"이며, 가중치만 남겨 둔다. 
            # 예측이 틀린 경우 error 값은 "1" 또는 "-1"이고 다음과 같이 가중치를 업데이트 한다.
            # weights[feature_index] += learning_rate * error * feature_value
            error = label - dp 

            # 예측이 틀린 경우 퍼셉트론은 다음과 같이 가중치를 업데이트한다.
            if error != 0:
                error_counter += 1
                # Updating the weights
                for index, value in features:
                    weights[index] += opts["learning_rate"] * error * log(1.+value)

        #Reporting stuff
        print("%s\t\t%s\t\t%s\t\t%s\t\t%s" % ( \
            pass_nr+1,
            error_counter,
            round(1 - error_counter /float(e+1),5),
            e+1,datetime.now()-start))

        #Oh heh, we have overfit :)
        if error_counter == 0 or error_counter < opts["errors_satisfied"]:
            print("%s errors found during training, halting"%error_counter)
            break
    return weights

In [6]:
def test_tron(loc_dataset,weights,opts):
    """
        output:
                preds: list, a list with
                [id,prediction,dotproduct,0-1normalized dotproduct]
    """
    start = datetime.now()
    print("\nTesting online\nErrors\t\tAverage\t\tNr. Samples\tSince Start")
    preds = []
    error_counter = 0
    for e, (label, id, features) in enumerate( \
        get_data_tsv(loc_dataset,opts) ):

        dotp = dot_product(features, weights)
        # 내적이 0.5보다 크다면 긍정으로 예측한다.
        dp = dotp > 0.5
        if dp > 0.5: # we predict positive class
            preds.append( [id, 1, dotp ] )
        else:
            preds.append( [id, 0, dotp ] )

        # get_data_tsv에서 테스트 데이터의 레이블을 1로 초기화 해주었음
        if label - dp != 0:
            error_counter += 1

    print("%s\t\t%s\t\t%s\t\t%s" % (
        error_counter,
        round(1 - error_counter /float(e+1),5),
        e+1,
        datetime.now()-start))

    # normalizing dotproducts between 0 and 1 
    # 내적을 구해 0과 1로 일반화 한다.
    # TODO: proper probability (bounded sigmoid?), 
    # online normalization
    max_dotp = max(preds,key=itemgetter(2))[2]
    min_dotp = min(preds,key=itemgetter(2))[2]
    for p in preds:
        # appending normalized to predictions
        # 정규화 된 값을 마지막에 추가해 준다.
        # (피처와 가중치에 대한 내적값 - 최소 내적값) / 최대 내적값 - 최소 내적값
        # 이 값이 캐글에서 0.95의 AUC를 얻을 수 있는 값이다.
        p.append((p[2]-min_dotp)/float(max_dotp-min_dotp)) 

    #Reporting stuff
    print("Done testing in %s"%str(datetime.now()-start))
    return preds

In [10]:
#Setting options
opts = {}
opts["D"] = 2 ** 25
opts["learning_rate"] = 0.1
opts["n_passes"] = 80 # Maximum number of passes to run before halting
opts["errors_satisfied"] = 0 # Halt when training errors < errors_satisfied
opts["random_init"] = False # set random weights, else set all 0
opts["clean"] = True # clean the text a little
opts["2grams"] = True # add 2grams

#training and saving model into weights
%time weights = train_tron("labeledTrainData.tsv",opts)


Pass		Errors		Average		Nr. Samples	Since Start
1		5681		0.77276		25000		0:00:24.138970
2		3120		0.8752		25000		0:00:46.635417
3		2217		0.91132		25000		0:01:09.363049
4		1693		0.93228		25000		0:01:33.994555
5		1256		0.94976		25000		0:01:58.705937
6		983		0.96068		25000		0:02:25.440292
7		785		0.9686		25000		0:02:47.545708
8		657		0.97372		25000		0:03:06.814161
9		533		0.97868		25000		0:03:25.614445
10		456		0.98176		25000		0:03:43.297138
11		402		0.98392		25000		0:04:02.784530
12		340		0.9864		25000		0:04:21.439342
13		238		0.99048		25000		0:04:39.803828
14		210		0.9916		25000		0:04:58.751760
15		166		0.99336		25000		0:05:18.760272
16		184		0.99264		25000		0:05:36.877885
17		164		0.99344		25000		0:05:55.011624
18		96		0.99616		25000		0:06:13.289440
19		155		0.9938		25000		0:06:32.111081
20		154		0.99384		25000		0:06:50.293952
21		88		0.99648		25000		0:07:11.175302
22		76		0.99696		25000		0:07:37.277693
23		74		0.99704		25000		0:07:59.942804
24		58		0.99768		25000		0:08:20.947278
25		67

In [11]:
# testing and saving predictions into preds
%time preds = test_tron("testData.tsv",weights,opts)



Testing online
Errors		Average		Nr. Samples	Since Start
12462		0.50152		25000		0:00:19.855956
Done testing in 0:00:19.881894
CPU times: user 18.5 s, sys: 150 ms, total: 18.7 s
Wall time: 19.9 s


In [12]:
preds[:10]

[['"12311_10"', 1, 83.31629110330545, 0.6389283539679332],
 ['"8348_2"', 0, -92.95103691308856, 0.4668831608145605],
 ['"5828_4"', 1, 2.1487562597357783, 0.5597050267234978],
 ['"7186_2"', 0, -6.654212933375497, 0.5511129152290115],
 ['"12128_7"', 1, 31.60751143353347, 0.588458155740479],
 ['"2913_8"', 1, 67.6511648226505, 0.6236384547730207],
 ['"4396_1"', 0, -42.90581047666061, 0.5157296529328205],
 ['"395_2"', 0, -24.67603962793411, 0.5335227657127406],
 ['"10616_1"', 0, -93.15898106725652, 0.4666801975509112],
 ['"9074_9"', 0, -26.131648707110006, 0.5321020228671955]]

In [13]:
# writing kaggle submission
# 캐글 점수 제출을 위한 서브미션 파일을 작성한다.
with open("submit_perceptron.csv","wb") as outfile:
    outfile.write('"id","sentiment"\n'.encode('utf-8'))
    for p in sorted(preds):
        outfile.write("{},{}\n".format(p[0],p[3]).encode('utf-8'))

In [14]:
import pandas as pd

presult = pd.DataFrame(preds)
presult.head()

,0,1,2,3
0,"""12311_10""",1,83.316291,0.638928
1,"""8348_2""",0,-92.951037,0.466883
2,"""5828_4""",1,2.148756,0.559705
3,"""7186_2""",0,-6.654213,0.551113
4,"""12128_7""",1,31.607511,0.588458


In [15]:
output_sentiment = presult[1].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

-76


1    12538
0    12462
Name: 1, dtype: int64

In [18]:
import numpy as np


array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,
        3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ])